In [6]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [7]:
print pd.__version__

0.18.1


In [8]:
os.chdir("C:\\Users\\Biswanath\\CodeForPrinceton\\AirQuality\\Data\\2016")
o3 = pd.read_csv('hourly_44201_2016.zip', compression='zip', sep=',', quotechar='"')
#o3 = pd.read_csv('hourly_44201_2016.csv', sep=',', quotechar='"')
print o3.head()

C:\Users\Biswanath\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


   State Code  County Code  Site Num  Parameter Code  POC  Latitude  \
0           1           49      9991           44201    1    34.289   
1           1           49      9991           44201    1    34.289   
2           1           49      9991           44201    1    34.289   
3           1           49      9991           44201    1    34.289   
4           1           49      9991           44201    1    34.289   

   Longitude  Datum Parameter Name  Date Local         ...           \
0  -85.97007  NAD83          Ozone  2016-01-01         ...            
1  -85.97007  NAD83          Ozone  2016-01-01         ...            
2  -85.97007  NAD83          Ozone  2016-01-01         ...            
3  -85.97007  NAD83          Ozone  2016-01-01         ...            
4  -85.97007  NAD83          Ozone  2016-01-01         ...            

    Units of Measure    MDL Uncertainty  Qualifier Method Type  Method Code  \
0  Parts per million  0.005         NaN        NaN         FEM     

In [9]:
print o3.iloc[0]


State Code                                       1
County Code                                     49
Site Num                                      9991
Parameter Code                               44201
POC                                              1
Latitude                                    34.289
Longitude                                 -85.9701
Datum                                        NAD83
Parameter Name                               Ozone
Date Local                              2016-01-01
Time Local                                   00:00
Date GMT                                2016-01-01
Time GMT                                     06:00
Sample Measurement                           0.019
Units of Measure                 Parts per million
MDL                                          0.005
Uncertainty                                    NaN
Qualifier                                      NaN
Method Type                                    FEM
Method Code                    

In [10]:
nj = o3[o3['State Name']=='New Jersey']


In [11]:
print nj.iloc[:25][['Latitude','Longitude','Date Local','Time Local','Sample Measurement']]
#print nj['Latitude','Longitude','Date Local','Time Local','Sample Measurement'][0]

        Latitude  Longitude  Date Local Time Local  Sample Measurement
970585   40.3125   -74.8729  2016-01-01      00:00               0.029
970586   40.3125   -74.8729  2016-01-01      01:00               0.028
970587   40.3125   -74.8729  2016-01-01      03:00               0.025
970588   40.3125   -74.8729  2016-01-01      04:00               0.023
970589   40.3125   -74.8729  2016-01-01      05:00               0.022
970590   40.3125   -74.8729  2016-01-01      06:00               0.023
970591   40.3125   -74.8729  2016-01-01      07:00               0.023
970592   40.3125   -74.8729  2016-01-01      08:00               0.025
970593   40.3125   -74.8729  2016-01-01      09:00               0.028
970594   40.3125   -74.8729  2016-01-01      10:00               0.029
970595   40.3125   -74.8729  2016-01-01      11:00               0.029
970596   40.3125   -74.8729  2016-01-01      12:00               0.031
970597   40.3125   -74.8729  2016-01-01      13:00               0.031
970598

In [12]:
#nj.iloc[0]['Date Local'],'Time Local','Sample Measurement']]

In [14]:
nj['DateStr'] = nj['Date Local'].dt.strftime('%Y%m%d')

AttributeError: Can only use .dt accessor with datetimelike values

In [ ]:
plt.plot(nj['Sample Measurement'])

In [ ]:
plt.show()